In [1]:
import data_load as dl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
path = r"C:\Users\Patrick\Datasets\sr\SR"
trial_paths = dl.combine_data(path)
trials = dl.generate_trial_objects(trial_paths)
processed_trials = dl.process_trial_data(trials)

C:\Users\Patrick\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#Smallest motion sample is 17559
# All G samples are 3030 long
# Take 2880 points from middle of 3030
# 3030 - 2880 = 150
#150/2 = 75
#Start each (Gs and downsampled motion on point 75) then sample out 2880

In [46]:
def synch_streams(G3, motion):
    #G2 = G2[75:-75]
    G3 = G3[75:-75]
    #G4 = G4[75:-75]
    #G5 = G5[75:-75]
    
    new_motion = motion[0:-1:6]
    new_motion = motion[75:(75+2880)]
    
    return G3, new_motion

In [47]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(100,12))
# ax1.plot(G2)
# ax2.plot(G3)
# ax3.plot(G4)
# ax4.plot(G5)
# ax5.plot(new_motion)

#Full segment is now 2880 samples for all streams at 10Hz --> 288 Seconds --> 4m48s

In [81]:
def generate_segments(G3, motion):
    full_run_length = 2880
    num_segments = 200
    indices = np.random.randint(0,(full_run_length - 100), size=(num_segments))
    
    G3_out = []
    motion_out = []
    
    for i in indices:
        #G2_seg = G2[i:(i+100)]/np.amax(G2[i:(i+100)])
        G3_seg = G3[i:(i+100)]/np.amax(G3[i:(i+100)])
        #G4_seg = G4[i:(i+100)]/np.amax(G4[i:(i+100)])
        #G5_seg = G5[i:(i+100)]/np.amax(G5[i:(i+100)])
        motion_seg = motion[i:(i+100)]/np.amax(motion[i:(i+100)])
        
        G3_out.append(G3_seg)
        motion_out.append(motion_seg)

    return G3_out, motion_out

In [49]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(15,12))
# ax1.plot(segments[148][0])
# ax2.plot(segments[148][1])
# ax3.plot(segments[148][2])
# ax4.plot(segments[148][3])
# ax5.plot(segments[148][4])

In [50]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(15,12))
# ax1.plot(segments[148][0][0:50])
# ax2.plot(segments[148][1][0:50])
# ax3.plot(segments[148][2][0:50])
# ax4.plot(segments[148][3][0:50])
# ax5.plot(segments[148][4][0:50])

In [94]:
#Start experimenting with NNet relationship between G3 and Net Motion
#Generate a bunch of G3/net motion samples:

trial = processed_trials[5]

G3_temp, motion_temp = synch_streams(trial.G3_AVG, trial.Motion_Conv)
G3, motion = generate_segments(G3_temp, motion_temp)


In [97]:
X = np.asarray(G3)
y = np.asarray(motion)

In [96]:
X.shape

(200, 100)

In [98]:
y.shape

(200, 100)